## 🚀 DipDetectorML – Dashboard & Integration Notebook  
**Team WaveRiders**  

This notebook shows the *presentation and integration side* of our project.  
This notebook shows the *presentation and integration side* of our project.  
- 📉 Live BTC monitoring (via CoinGecko API)  
- 🤖 Machine learning predictions (Random Forest from Mohammed’s notebook)  
- 📧 Email alert previews (HTML) with user thresholds  
- ☁️ AWS SES cloud setup mock-up (scaffold only)  
- 📈 Monthly risk display (single probability from the ML model output)  
- 📊 Visuals: price line + dip line, last checks table  
- 🖥️ Streamlit app code (`app.py`) to run a demo dashboard  

**Author & Role:** Jessenia — Project Lead, Dashboard & Integration  
**Goal:** Give stakeholders a product-like demo that ties live data, ML output, and alert UX together.

---

## 🛠️ 2. Imports & Dependencies

In [ ]:

# Core
import os, json
from datetime import datetime, timezone

# Data & Viz
import pandas as pd
import numpy as np
import plotly.express as px
from IPython.display import HTML, display

# Networking
import requests

# Dashboard (used in app.py; not executed inside this notebook)
import streamlit as st  # ok to import even if not run here

# Optional AWS (only if you installed boto3 above)
# import boto3
# from dotenv import load_dotenv

## 📑 3. Data Dictionary

In [9]:
## 📑 Data Dictionary — Display Layer (Jessenia)

**Live market snapshot**  
- `timestamp_utc` — time the snapshot was pulled  
- `symbol` — “BTC”  
- `price_usd` — latest price shown  
- `change_24h_pct` — 24-hour percent change (fraction, e.g., −0.052 = −5.2%)  
- `change_7d_pct` — 7-day percent change (optional)

**Alert configuration (user-facing)**  
- `threshold_pct` — selected alert threshold (2, 5, 10, 15, 20)  
- `window` — lookback used for dip check (“24h” or “7d”)  
- `triggered` — True/False if threshold crossed at this check  
- `email_preview_html` — rendered alert HTML (for preview)  
- `email_to` — demo recipient (test value)

**ML summary (display-only)**  
- `monthly_dip_prob` — probability of ≥20% monthly dip (0–1) from Mohammed’s output  
- `model_version` — e.g., “rf_monthly_v1.pkl” (optional)  
- `last_model_update` — when the probability was produced (optional)


SyntaxError: invalid character '—' (U+2014) (3228636934.py, line 4)

## Live BTC Snapshot

In [7]:
def fetch_btc_snapshot():
    """Live price + 24h change from CoinGecko. Falls back to demo values if rate-limited."""
    url = "https://api.coingecko.com/api/v3/simple/price"
    params = {"ids": "bitcoin", "vs_currencies": "usd", "include_24hr_change": "true"}
    try:
        r = requests.get(url, params=params, timeout=10)
        r.raise_for_status()
        data = r.json()["bitcoin"]
        price = float(data["usd"])
        ch24 = float(data.get("usd_24h_change", 0.0)) / 100.0  # convert %→fraction
    except Exception:
        # Fallback demo numbers if API is blocked / rate-limited
        price, ch24 = 56800.00, -0.034  # -3.4% demo
    now = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M UTC")
    return {"timestamp_utc": now, "symbol": "BTC", "price_usd": price, "change_24h_pct": ch24, "change_7d_pct": None}

snap = fetch_btc_snapshot()
snap


{'timestamp_utc': '2025-08-28 16:31 UTC',
 'symbol': 'BTC',
 'price_usd': 113026.0,
 'change_24h_pct': 0.011320133766837022,
 'change_7d_pct': None}

## Markdown: Email Alert Preview

In [8]:
def fmt_usd(x): return f"${x:,.2f}"
def fmt_pct(x): return f"{x*100:.2f}%"

def build_subject(symbol, pct_change, price, window, threshold):
    arrow = "▼" if pct_change < 0 else "▲"
    return f"✅ DipDetectorML – {symbol} Alert: {arrow}{fmt_pct(pct_change)} in last {window} (Now {fmt_usd(price)})"

def build_html(symbol, price_now, pct_change, threshold, window, monthly_prob=None):
    prob_line = f"<li>Model forecast: <b>{monthly_prob*100:.1f}%</b> chance of ≥20% dip this month</li>" if monthly_prob is not None else ""
    color = "#d32f2f" if pct_change < 0 else "#2e7d32"
    arrow = "▼" if pct_change < 0 else "▲"
    return f"""
<!doctype html>
<html>
  <body style="font-family:Arial,sans-serif;line-height:1.55;color:#111;margin:0;padding:20px;background:#f6f8fb">
    <div style="max-width:640px;margin:0 auto;background:#fff;border-radius:12px;padding:20px;box-shadow:0 2px 8px rgba(0,0,0,.07)">
      <h2 style="color:#e85d04;margin:0 0 10px">🚀 DipDetectorML Alert – {int(threshold)}% Dip Triggered!</h2>
      <div style="display:inline-block;padding:6px 10px;border-radius:999px;background:{color};color:#fff;font-weight:700;margin:6px 0 14px">
        {arrow} {fmt_pct(pct_change)} in last {window}
      </div>
      <ul style="margin-top:0">
        <li>Current price: <b>{fmt_usd(price_now)}</b></li>
        {prob_line}
      </ul>
      <p>📌 Example playbook for accumulators:</p>
      <ul>
        <li>5% dip → advance weekly DCA</li>
        <li>10% dip → 2× DCA</li>
        <li>20%+ dip → 3× DCA (if cash reserved)</li>
      </ul>
      <hr style="border:none;border-top:1px solid #eee;margin:16px 0">
      <p style="font-size:12px;color:#666">You’re seeing this as a preview in Jessenia’s notebook (no email sent).</p>
    </div>
  </body>
</html>
""".strip()

# Choose threshold + window for the preview
threshold = 10
window = "24h"
symbol  = snap["symbol"]; price = snap["price_usd"]; ch24 = snap["change_24h_pct"]

# Until Mohammed exports a live artifact, use a demo probability
monthly_prob = 0.32

subject = build_subject(symbol, ch24, price, window, threshold)
html_preview = build_html(symbol, price, ch24, threshold, window, monthly_prob)

print("Subject:", subject)
display(HTML(html_preview))


Subject: ✅ DipDetectorML – BTC Alert: ▲1.13% in last 24h (Now $113,026.00)


NameError: name 'HTML' is not defined